In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import sys
from pathlib import Path

parent = str(Path().absolute().parent)
if parent not in sys.path:
    sys.path.insert(0,parent)
from sherlock.evidence_store import EvidenceStore
from sherlock.models import Claim, Evidence, Argument
from sherlock.agents import ClaimSupportAgent

In [3]:
store = EvidenceStore("wally_evidence")

# Add our evidence
evidence_ids = []
evidence_ids.append(store.add_evidence("Librarian Jane saw someone matching Wally's description at 3pm"))
evidence_ids.append(store.add_evidence("Security camera shows someone in red and white stripes in the park"))
evidence_ids.append(store.add_evidence("Library card records don't show Wally checking out any books today"))
evidence_ids.append(store.add_evidence("Wally's phone GPS shows he was at the library coordinates"))



In [10]:


agent = ClaimSupportAgent(store)

# Create a test claim
wally_claim = Claim(text="Wally is in the library")


In [11]:
dc = agent.query_evidence('wally library')

16:42:18 | INFO | Querying evidence store with: wally library


In [12]:
dc.model_dump()

{'evidence': [{'id': 'ev_4',
   'text': "Wally's phone GPS shows he was at the library coordinates"},
  {'id': 'ev_1',
   'text': "Librarian Jane saw someone matching Wally's description at 3pm"},
  {'id': 'ev_3',
   'text': "Library card records don't show Wally checking out any books today"}]}

In [13]:

# Evaluate it
updated_claim = agent.evaluate_claim(wally_claim)

16:42:31 | INFO | Starting evaluation of claim: Wally is in the library
16:42:31 | INFO | Starting iteration 1
16:42:45 | INFO | Tool_use stop reason. Response.content: [TextBlock(citations=None, text='<thinking>\nTo evaluate the claim "Wally is in the library", I will need to find supporting evidence using the available tools.\n\nThe most relevant tool is query_evidence, which allows searching for evidence using keywords. I can use keywords from the claim like "Wally" and "library" to search for relevant evidence.\n\nOnce I have 2-3 pieces of supporting evidence, I can use the create_argument tool to construct an argument. This tool requires the following parameters:\n- text: The main text of the argument. I can summarize the key points from the evidence to form the argument text.\n- supports: Whether the argument supports (True) or opposes (False) the claim. Since my goal is to find supporting evidence, this will be set to True. \n- evidence_collection: The evidence supporting the ar

In [15]:
updated_claim.model_dump()

{'text': 'Wally is in the library',
 'id': 'wally-is-in-the-library',
 'arguments': [{'id': '18327231-f4c7-42b8-bdac-ac60cfd32c01',
   'text': "The evidence indicates Wally was likely at the library. His phone's GPS data places him at the library coordinates, providing strong support for the claim. Additionally, librarian Jane reports seeing someone matching Wally's description at the library around 3pm, further corroborating his presence there, although this eyewitness account leaves some room for uncertainty.",
   'supports': True,
   'evidence_collection': {'evidence': [{'id': 'ev_4',
      'text': "Wally's phone GPS shows he was at the library coordinates"},
     {'id': 'ev_1',
      'text': "Librarian Jane saw someone matching Wally's description at 3pm"}]},
   'subclaims': []}],
 'likelihood': {'supporting': 2, 'opposing': 0}}

In [23]:
updated_claim.likelihood

Likelihood(supporting=2, opposing=0)

In [ ]:

# 3. Create a claim
wally_claim = Claim(text="Wally is in the library")
print(f"Created claim: {wally_claim.text} (ID: {wally_claim.id})")

# 4. Query for relevant evidence
query_results = store.query("Wally library location", n_results=3)
print("\nRelevant evidence for query 'Wally library location':")
for i, result in enumerate(query_results):
    print(f"{i+1}. {result['text']} (ID: {result['id']})")

selected_evidence_id = query_results[1]['id']

# Create an argument manually
argument = Argument(
    text="Based on visual identification, Wally is at the library",
    supports=True,  # This supports our claim
    evidence_ids=[selected_evidence_id]
)

# Add to our claim
wally_claim.arguments.append(argument)
wally_claim._update_likelihood()



In [4]:
argument.model_dump()

{'id': 'ed9ce919-9d99-42ad-a96e-e76843934bae',
 'text': 'Based on visual identification, Wally is at the library',
 'supports': True,
 'evidence_ids': ['ev_1'],
 'subclaim_ids': []}

In [5]:
store.collection.get('ev_1')

{'ids': ['ev_1'],
 'embeddings': None,
 'documents': ["Librarian Jane saw someone matching Wally's description at 3pm"],
 'uris': None,
 'data': None,
 'metadatas': [{'type': 'evidence'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [3]:
wally_claim.id

'wally-is-in-the-library'